## Decorators

The question I kept repeating while going through so many decorator guides and blogs was why do we need them. Basically decorators allow you to write modular code by making it possible to add or wrap functionality to/around existing functions without changing them.
OK! but WHY do we need the decorator syntax? Why do we need nested functions to create a decorator? Why can't we just take an existing function, write another one with some extra functionality, and feed the first to the 2nd function to make it do something else?

Source: https://stackoverflow.com/questions/52593649/what-is-the-purpose-of-decorators-why-use-them

The current method of applying a transformation to a function or method places the actual transformation after the function body. For large functions this separates a key component of the function's behavior from the definition of the rest of the function's external interface. 
This becomes less readable with longer methods. It also seems less than pythonic to name the function three times for what is conceptually a single declaration. A solution to this problem is to move the transformation of the method closer to the method's own declaration. The intent of the new syntax is to replace

```python
def foo(cls):
    pass 
foo = synchronized(lock)(foo) 
foo = classmethod(foo)
```

with

```python
@classmethod
@synchronized(lock)
def foo(cls):
    pass
```